In [14]:
import pandas as pd
import json
import os
import re

In [15]:
pickle_files = [
    "./data/sonnet3.pkl",
    "./data/sonnet3.5-3.pkl",
    "./data/gpt-4o-3.pkl",
    "./data/gemini-1.5-pro-3.pkl",
]

dfs = [pd.read_pickle(file) for file in pickle_files]
df1 = pd.concat(dfs)

df1["prompt"] = "How many times do the blue and red line plots cross each other?"


pickle_files = [
    "./data/sonnet2.pkl",
    "./data/sonnet3.5-2.pkl",
    "./data/gpt-4o-2.pkl",
    "./data/gemini-1.5-pro-2.pkl",
]

dfs = [pd.read_pickle(file) for file in pickle_files]
df2 = pd.concat(dfs)

df2["prompt"] = "How many times do the blue and red lines intersect?"

In [16]:
df = pd.concat([df1, df2])

In [17]:
df

,image_path,num_points,num_intersections,line_colors,line_thickness,model_output_raw,extracted_number,is_correct,Model,prompt
0,image_0_thickness_2.png,3,1,"[red, blue]",2,The blue and red line segments intersect or cr...,1,True,Sonnet,How many times do the blue and red line plots ...
1,image_0_thickness_3.png,3,1,"[red, blue]",3,The blue and red line segments intersect or cr...,1,True,Sonnet,How many times do the blue and red line plots ...
2,image_0_thickness_4.png,3,1,"[red, blue]",4,The blue and red line segments cross each othe...,1,True,Sonnet,How many times do the blue and red line plots ...
3,image_1_thickness_2.png,3,2,"[red, blue]",2,The blue and red line plots cross each other a...,1,False,Sonnet,How many times do the blue and red line plots ...
4,image_1_thickness_3.png,3,2,"[red, blue]",3,The blue and red line segments intersect or cr...,1,False,Sonnet,How many times do the blue and red line plots ...
...,...,...,...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,The blue and red lines intersect **twice**. \n,2,True,Gemini-1.5-Pro,How many times do the blue and red lines inter...
146,image_48_thickness_4.png,3,2,"[red, blue]",4,The blue and red lines intersect **twice**. \n,2,True,Gemini-1.5-Pro,How many times do the blue and red lines inter...
147,image_49_thickness_2.png,3,0,"[red, blue]",2,The blue and red lines do not intersect in the...,0,True,Gemini-1.5-Pro,How many times do the blue and red lines inter...
148,image_49_thickness_3.png,3,0,"[red, blue]",3,The blue and red lines intersect **zero** time...,0,True,Gemini-1.5-Pro,How many times do the blue and red lines inter...


In [18]:
900 / 3

300.0

In [19]:
accuracy_per_model_and_word = df.groupby(["Model"])["is_correct"].mean()

# Convert the accuracy to percentage and print the result
accuracy_per_model_and_word = accuracy_per_model_and_word * 100
# round 2 decimal places
accuracy_per_model_and_word = accuracy_per_model_and_word.round(2)

accuracy_per_model_and_word.T.reset_index()

,Model,is_correct
0,Gemini-1.5-Pro,73.33
1,Sonnet,49.67
2,Sonnet-3.5,52.67
3,gpt-4o,47.16


Model
Gemini-1.5-Pro    77.33
Sonnet            49.33
gpt-4o            46.31
Name: is_correct, dtype: float64

In [20]:
# Group by both Model and line_thickness
accuracy_per_model_thickness = df.groupby(["Model", "line_thickness"])[
    "is_correct"
].mean()

# Convert the accuracy to percentage
accuracy_per_model_thickness = accuracy_per_model_thickness * 100
# Round to 2 decimal places
accuracy_per_model_thickness = accuracy_per_model_thickness.round(2)

accuracy_per_model_thickness.reset_index()

,Model,line_thickness,is_correct
0,Gemini-1.5-Pro,2,72.00
1,Gemini-1.5-Pro,3,71.00
2,Gemini-1.5-Pro,4,77.00
3,Sonnet,2,44.00
4,Sonnet,3,49.00
5,Sonnet,4,56.00
6,Sonnet-3.5,2,51.00
7,Sonnet-3.5,3,52.00
8,Sonnet-3.5,4,55.00
9,gpt-4o,2,47.47
